Задача 1. Объединить списки и проссумировать значения в столбцах по id клиентов

Вывод списка #1

In [2]:
import os
import glob
import pandas as pd
import numpy as np
from glob import glob
df1 = pd.read_csv("pytest_1.csv", sep = ';')
df1

,client_id,2017,2018
0,1,48,15
1,2,85,12
2,3,54,11
3,5,1,2


Вывод списка #2

In [3]:
df2 = pd.read_csv("pytest_2.csv", sep = ';')
df2

,client_id,2018,2019,2020
0,1,NaN,44,84
1,2,100.0,76,0
2,3,NaN,86,68


Сортировка файлов со списками 

In [4]:
from glob import glob
source_files = sorted(glob('pytest_*.csv'))
source_files

['pytest_1.csv', 'pytest_2.csv']

Соединение списков #1 и #2

In [5]:
a = pd.concat((pd.read_csv(file, sep = ';').assign(filename = file)
          for file in source_files))
a

/Users/Kurenkov/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  


,2017,2018,2019,2020,client_id,filename
0,48.0,15.0,NaN,NaN,1,pytest_1.csv
1,85.0,12.0,NaN,NaN,2,pytest_1.csv
2,54.0,11.0,NaN,NaN,3,pytest_1.csv
3,1.0,2.0,NaN,NaN,5,pytest_1.csv
0,NaN,NaN,44.0,84.0,1,pytest_2.csv
1,NaN,100.0,76.0,0.0,2,pytest_2.csv
2,NaN,NaN,86.0,68.0,3,pytest_2.csv


Группировка списков по id-клиентов

In [6]:
group_by_id = a.groupby(['client_id'])[[str(y) for y in range(2017, 2021)]].sum()

In [15]:
print(group_by_id)

           2017   2018  2019  2020
client_id                         
1          48.0   15.0  44.0  84.0
2          85.0  112.0  76.0   NaN
3          54.0   11.0  86.0  68.0
5           1.0    2.0   NaN   NaN


In [16]:
group_by_id.to_csv('group_by_id.csv')

Задача 2. Посчитать средний оборот за предыдущие 2 года для каждого клиента

In [19]:
group_by_id = group_by_id.replace(0, np.nan) #меняем нули на NaN, чтобы правильно считались средние
group_by_id

,2017,2018,2019,2020
client_id,,,,
1,48.0,15.0,44.0,84.0
2,85.0,112.0,76.0,NaN
3,54.0,11.0,86.0,68.0
5,1.0,2.0,NaN,NaN


In [75]:
avg2years = group_by_id.rolling(window=2, min_periods=2, axis=1).mean() #считаем среднее за последние 2 года
#avg2years.fillna(0)
avg2years

,2017,2018,2019,2020
client_id,,,,
1,NaN,31.5,29.5,64.0
2,NaN,98.5,94.0,NaN
3,NaN,32.5,48.5,77.0
5,NaN,1.5,NaN,NaN


In [76]:
#выставляем категории клиентов
limit_0 = 0
limit_1 = 10
limit_2 = 50
limit_3 = 90

filter_cat = [0,1,2,3,4]
filter_val = [-1, limit_0, limit_1, limit_2, limit_3, np.inf]


In [77]:
#проходимся по таблице и проставляем клиентам категории в зависимости от среднего значения 
for col in avg2years:
    avg2years[col] = pd.cut(avg2years[col],
                   bins = filter_val,
                   labels = filter_cat
                    )
avg2years

,2017,2018,2019,2020
client_id,,,,
1,NaN,2,2.0,3.0
2,NaN,4,4.0,NaN
3,NaN,2,2.0,3.0
5,NaN,1,NaN,NaN
